<!-- NFL Big Data Bowl 2025  -->
<a id="0"></a>
<div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#FF6B6B; font-size:300%; text-align:left; padding:3.0px; background: #2B303A; border-bottom: 8px solid #36454F">
    NFL Big Data Bowl 2025 | Automated EDA - Sweetviz - Langchain <br>
</div>

<img src="https://i.imgur.com/Yvo4uz6.jpg" width="500px">
Photo de <a href="https://unsplash.com/fr/@aussiedave?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Dave Adamson</a> sur <a href="https://unsplash.com/fr/photos/un-gros-plan-dun-objet-en-cuir-avec-une-banane-dessus-EnX2W4WtHLg?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Unsplash</a>

# Challenge overview

## Competition

- **Competition Overview:** The NFL Big Data Bowl 2025 challenges participants to analyze pre-snap player tracking data to predict and understand offensive and defensive tendencies in football. Using Next Gen Stats data, participants are asked to identify patterns and generate actionable insights into team and player behavior from pre-snap to post-snap.

- **Data and Objective:** The competition uses player tracking data from the NFL, which includes player locations, movements, and configurations before the snap. The goal is to generate creative metrics or predictions, such as play type, player assignments, or defensive schemes, that can help NFL teams gain strategic advantages.

- **Tracks and Evaluation:** Participants can submit to three different tracks: Undergraduate (for students), Metric (developing performance metrics), or Coaching Presentation (scouting-focused analysis). Submissions are evaluated on Football applicability, Data Science rigor, Report quality, and Visualization effectiveness, with cash prizes for the top 10 teams and the chance to present at the 2025 NFL Scouting Combine.

## Notebook aim

In this notebook, I am experimenting with automated EDA tools to see if I can get quick insights into the datasets.

I am using Sweetviz to generate quick data visualizations, and LangChain and GPT to generate quick data reports.

I aim to create another classical EDA notebook in parallel to explore the data in more detail.

In [1]:
!pip install langchain-core > /dev/null 2>&1
!pip install langchain-openai  > /dev/null 2>&1
!pip install sweetviz > /dev/null 2>&1

In [2]:
import numpy as np
import pandas as pd
import sweetviz as sv

# LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import json

from IPython.display import Markdown, display

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
OPENAI_API_KEY = user_secrets.get_secret("openai_key")

In [4]:
# Define the LLM model using LangChain
model = ChatOpenAI(
    model='gpt-4o-2024-05-13',
    temperature=0,
    api_key=OPENAI_API_KEY
)

In [5]:
# Define the prompt template for LangChain
template_eda = """Provide an analysis of the following EDA summary:
{context}

Key insights and observations:
"""

In [6]:
# Function to classify columns into continuous and categorical
def classify_columns(df):
    continuous_cols = []
    categorical_cols = []
    for column in df.columns:
        if df[column].dtypes == 'object':
            categorical_cols.append(column)
        else:
            unique_values = df[column].nunique()
            if unique_values < 15:
                categorical_cols.append(column)
            else:
                continuous_cols.append(column)
    return continuous_cols, categorical_cols

# Function to perform basic visualizations for continuous and categorical features
def eda_visualizations(df, target=None):
    continuous_cols, categorical_cols = classify_columns(df)
    
    # Plotting continuous columns
    for col in continuous_cols:
        plt.figure(figsize=(10, 4))
        sns.histplot(df[col], kde=True)
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.show()
    
    # Plotting categorical columns
    for col in categorical_cols:
        plt.figure(figsize=(10, 4))
        sns.countplot(data=df, x=col, hue=target)
        plt.title(f'Count plot for {col}')
        plt.xlabel(col)
        plt.ylabel('Count')
        plt.xticks(rotation=45)
        plt.show()

# Function to compare train and test datasets
def compare_train_test(train, test):
    continuous_cols, categorical_cols = classify_columns(train)
    
    # Compare continuous columns
    for col in continuous_cols:
        plt.figure(figsize=(10, 4))
        sns.kdeplot(train[col], label='Train', shade=True)
        sns.kdeplot(test[col], label='Test', shade=True)
        plt.title(f'Comparison of {col} Distribution in Train vs Test')
        plt.xlabel(col)
        plt.ylabel('Density')
        plt.legend()
        plt.show()
    
    # Compare categorical columns
    for col in categorical_cols:
        if col in test.columns:  # Ensure the column exists in the test dataset
            plt.figure(figsize=(10, 4))
            train_counts = train[col].value_counts(normalize=True)
            test_counts = test[col].value_counts(normalize=True)
            train_counts.plot(kind='bar', alpha=0.5, label='Train', color='blue')
            test_counts.plot(kind='bar', alpha=0.5, label='Test', color='red')
            plt.title(f'Comparison of {col} Proportions in Train vs Test')
            plt.xlabel(col)
            plt.ylabel('Proportion')
            plt.legend()
            plt.xticks(rotation=45)
            plt.show()
            
# Function to create key statistics for a dataset
def eda_summary(df):
    summary = {}
    
    # General Info
    summary['general'] = {
        'num_rows': df.shape[0],
        'num_columns': df.shape[1],
        'num_missing_values': df.isnull().sum().sum(),
        'percent_missing_values': df.isnull().mean().mean() * 100
    }
    
    # Column Data Types
    summary['data_types'] = df.dtypes.to_dict()
    
    # Missing Value Summary (per column)
    summary['missing_values'] = (
        df.isnull()
        .sum()
        .to_frame(name='missing_count')
        .assign(percent_missing=lambda x: (x['missing_count'] / df.shape[0]) * 100)
        .to_dict(orient='index')
    )
    
    # Numerical Summary (Mean, Median, Std, Min, Max)
    describe_df = df.describe()
    numerical_columns = ['mean', '50%', 'std', 'min', 'max']
    available_columns = [col for col in numerical_columns if col in describe_df.columns]
    summary['numerical_summary'] = (
        describe_df[available_columns]
        .rename(columns={'50%': 'median'})
        .to_dict(orient='index')
    )
    
    # Unique Counts for Categorical Columns
    summary['categorical_summary'] = (
        df.select_dtypes(include=['object', 'category'])
        .nunique()
        .to_frame(name='unique_counts')
        .to_dict(orient='index')
    )
    
    # Skewness and Kurtosis
    summary['skewness_kurtosis'] = {
        column: {
            'skewness': df[column].skew(),
            'kurtosis': df[column].kurt()
        } for column in df.select_dtypes(include=[np.number]).columns
    }
    
    # Correlations
    try:
        summary['correlations'] = df.corr(numeric_only=True).to_dict()
    except ValueError:
        summary['correlations'] = "Unable to calculate correlations due to data type issues."
    
    # Outlier Count based on IQR
    outlier_summary = {}
    for column in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]
        outlier_summary[column] = {
            'outlier_count': outliers.shape[0],
            'percent_outliers': (outliers.shape[0] / df.shape[0]) * 100
        }
    summary['outlier_summary'] = outlier_summary

    return summary

# Automated EDA

In [7]:
players = pd.read_csv("../input/nfl-big-data-bowl-2025/players.csv")
player_play = pd.read_csv("../input/nfl-big-data-bowl-2025/player_play.csv")
games = pd.read_csv("../input/nfl-big-data-bowl-2025/games.csv")
plays = pd.read_csv("../input/nfl-big-data-bowl-2025/plays.csv")

## Player EDA

In [8]:
players.head()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


In [9]:
# Automated EDA report with sweetviz
report = sv.analyze(players, pairwise_analysis="off")
report.show_html(filepath='/kaggle/working/players_report.html', open_browser=False)

                                             |          | [  0%]   00:00 -> (? left)

Report /kaggle/working/players_report.html was generated.


In [10]:
# Automated EDA with gpt

summary = eda_summary(players)

# Convert summary to JSON format
summary_json = json.dumps(summary, indent=4, default=str)

# Define the prompt template for LangChain
prompt = ChatPromptTemplate.from_template(template_eda)

# Create a chain to pass the summary to the model
chain = prompt | model | StrOutputParser()

# Invoke the chain to analyze the EDA summary
result = chain.invoke(summary_json)

# Display the result
display(Markdown(result))

Based on the provided Exploratory Data Analysis (EDA) summary, here are the key insights and observations:

### General Overview
1. **Dataset Size**:
   - The dataset contains 1,697 rows and 7 columns.
   - There are 487 missing values in total, which constitutes approximately 4.10% of the entire dataset.

### Data Types
2. **Data Types**:
   - The dataset includes a mix of data types:
     - `int64`: `nflId`, `weight`
     - `object`: `height`, `birthDate`, `collegeName`, `position`, `displayName`

### Missing Values
3. **Missing Values**:
   - The `birthDate` column has 487 missing values, which is 28.70% of the column. This is a significant proportion and may need to be addressed through imputation or other methods.
   - All other columns have no missing values.

### Categorical Summary
4. **Unique Counts**:
   - `height`: 16 unique values
   - `birthDate`: 989 unique values
   - `collegeName`: 225 unique values
   - `position`: 19 unique values
   - `displayName`: 1,687 unique values

   This indicates a high level of diversity in `birthDate`, `collegeName`, and `displayName`, while `height` and `position` have fewer unique values.

### Skewness and Kurtosis
5. **Skewness and Kurtosis**:
   - `nflId`:
     - Skewness: -0.433 (slightly left-skewed)
     - Kurtosis: -0.505 (platykurtic, indicating a flatter distribution)
   - `weight`:
     - Skewness: 0.433 (slightly right-skewed)
     - Kurtosis: -1.037 (platykurtic, indicating a flatter distribution)

### Correlations
6. **Correlations**:
   - `nflId` and `weight` have a weak negative correlation (-0.0896), suggesting that there is little to no linear relationship between these two variables.

### Outliers
7. **Outliers**:
   - `nflId` has 4 outliers, which is about 0.24% of the data for this column.
   - `weight` has no outliers.

### Key Insights and Observations
1. **Missing Data**:
   - The high percentage of missing values in the `birthDate` column is a concern and should be addressed. Depending on the analysis, you might consider imputation, exclusion, or further investigation into why these values are missing.

2. **Data Diversity**:
   - The high number of unique values in `birthDate`, `collegeName`, and `displayName` suggests a diverse dataset, which could be beneficial for certain types of analyses but may also require careful handling to avoid overfitting in predictive models.

3. **Distribution Characteristics**:
   - Both `nflId` and `weight` show slight skewness and are platykurtic, indicating that their distributions are relatively flat compared to a normal distribution. This might affect statistical analyses that assume normality.

4. **Weak Correlation**:
   - The weak correlation between `nflId` and `weight` suggests that these variables do not have a strong linear relationship, which might imply that `nflId` (likely a unique identifier) does not provide much information about `weight`.

5. **Outliers**:
   - The presence of outliers in `nflId` might need further investigation to understand if they are data entry errors or valid extreme values. The absence of outliers in `weight` suggests a more consistent data range for this variable.

### Recommendations
1. **Handle Missing Values**:
   - Address the missing values in the `birthDate` column through appropriate methods such as imputation or exclusion, depending on the context of the analysis.

2. **Further Analysis**:
   - Conduct further analysis on the outliers in `nflId` to determine their impact and validity.
   - Explore the categorical variables with high unique counts to understand their distribution and potential impact on the analysis.

3. **Data Transformation**:
   - Consider transforming the `weight` and `nflId` variables if normality is required for certain analyses, although their current skewness and kurtosis values are not extreme.

4. **Feature Engineering**:
   - Given the diversity in categorical variables, consider feature engineering techniques such as encoding or binning to make these variables more suitable for machine learning models.

By addressing these points, you can improve the quality and usability of the dataset for subsequent analyses.

## Try to identify missing birthdates

I am using GPT to fill in some missing birthdates, hoping that if they are available on the internet, they might be part of the training dataset of GPT-4. I added them in a separate column because I am still determining if this approach will work. I have begun manually verifying the results. Some of the added birthdates seem correct, but in other cases, only the year is accurate, while the day and month are not.

In [11]:
# Define the prompt template for LangChain
template_missing_birthdate = """Given the following details of an NFL player:
Player Name: {displayName}
Position: {position}
College: {collegeName}

Please provide only the player's birth date in the format YYYY-MM-DD, and nothing more."""

In [12]:
players_df = pd.DataFrame(players)

# Add a new column for inferred birthdates
players_df['inferredBirthDate'] = players_df['birthDate']

# Identify rows with missing birthDate
missing_birthdate_rows = players_df[players_df['birthDate'].isna()]

# Create a ChatPromptTemplate from the template
prompt = ChatPromptTemplate.from_template(template_missing_birthdate)

# Iterate over rows with missing birthDate to predict values using GPT
for idx, row in missing_birthdate_rows.iterrows():
    # Format the prompt with player details
    context = {
        'displayName': row['displayName'],
        'position': row['position'],
        'collegeName': row['collegeName'],
        'height': row['height'],
        'weight': row['weight']
    }
    
    # Create the prompt with filled-in context
    chain = prompt | model | StrOutputParser()

    # Invoke the chain to get the birthdate
    result = chain.invoke(context).strip()
    
    # Assign the result to the new inferredBirthDate column
    players_df.at[idx, 'inferredBirthDate'] = result

# Display the updated DataFrame
players_df

,nflId,height,weight,birthDate,collegeName,position,displayName,inferredBirthDate
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,1977-08-03
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,1982-01-22
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,1983-12-02
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis,1984-05-19
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan,1985-05-17
...,...,...,...,...,...,...,...,...
1692,55190,6-1,285,NaN,Maryland,DE,Sam Okuayinonu,1998-01-01
1693,55200,6-6,266,NaN,Indiana,DT,Ryder Anderson,1998-10-15
1694,55239,6-2,300,NaN,Pennsylvania,DT,Prince Emili,1998-04-09
1695,55240,6-1,185,NaN,Buffalo,CB,Ja'Marcus Ingram,1999-07-15


In [13]:
players_df.to_csv('/kaggle/working/players_with_inferred_birth_date.csv')

## Player Plays EDA

In [14]:
player_play.head()

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,...,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
0,2022090800,56,35472,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,2022090800,56,42392,BUF,0,0,0,0,0,0,...,NaN,NaN,47917.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2,2022090800,56,42489,BUF,0,0,0,0,0,1,...,1.0,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022090800,56,44875,BUF,0,0,0,0,0,0,...,NaN,NaN,43335.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,2022090800,56,44985,BUF,0,0,0,0,0,0,...,1.0,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
report = sv.analyze(player_play, pairwise_analysis="off")
report.show_html(filepath='/kaggle/working/player_play_report.html', open_browser=False)

                                             |          | [  0%]   00:00 -> (? left)

Report /kaggle/working/player_play_report.html was generated.


In [16]:
summary = eda_summary(player_play)
summary_json = json.dumps(summary, indent=4, default=str)
prompt = ChatPromptTemplate.from_template(template_eda)
chain = prompt | model | StrOutputParser()
result = chain.invoke(summary_json)
display(Markdown(result))

The provided Exploratory Data Analysis (EDA) summary offers a comprehensive overview of a dataset with 354,727 rows and 50 columns. Here are the key insights and observations:

### General Overview
- **Number of Rows**: 354,727
- **Number of Columns**: 50
- **Total Missing Values**: 5,188,268
- **Percentage of Missing Values**: 29.25%

### Data Types
- **Numerical Columns**: Predominantly `int64` and `float64`.
- **Categorical Columns**: `object` and `bool`.

### Missing Values
- **High Missing Values**: 
  - `penaltyNames` (99.89%)
  - `timeToPressureAsPassRusher` (98.78%)
  - `getOffTimeAsPassRusher` (86.46%)
  - `wasRunningRoute` and `routeRan` (87.94%)
  - `blockedPlayerNFLId3` (99.99%)
  - `pff_secondaryDefensiveCoverageMatchupNflId` (99.33%)

- **No Missing Values**: 
  - `gameId`, `playId`, `nflId`, `teamAbbr`, `hadRushAttempt`, `rushingYards`, `hadDropback`, `passingYards`, `sackYardsAsOffense`, `hadPassReception`, `receivingYards`, `wasTargettedReceiver`, `yardageGainedAfterTheCatch`, `fumbles`, `fumbleLost`, `fumbleOutOfBounds`, `assistedTackle`, `forcedFumbleAsDefense`, `halfSackYardsAsDefense`, `passDefensed`, `quarterbackHit`, `sackYardsAsDefense`, `safetyAsDefense`, `soloTackle`, `tackleAssist`, `tackleForALoss`, `tackleForALossYardage`, `hadInterception`, `interceptionYards`, `fumbleRecoveries`, `fumbleRecoveryYards`, `penaltyYards`, `causedPressure`.

### Categorical Summary
- **Unique Counts**:
  - `teamAbbr`: 32
  - `penaltyNames`: 30
  - `inMotionAtBallSnap`, `shiftSinceLineset`, `motionSinceLineset`: 2 each
  - `routeRan`: 12
  - `pff_defensiveCoverageAssignment`: 19

### Skewness and Kurtosis
- **High Skewness and Kurtosis**: 
  - `rushingYards`, `passingYards`, `sackYardsAsOffense`, `receivingYards`, `yardageGainedAfterTheCatch`, `fumbles`, `fumbleLost`, `fumbleOutOfBounds`, `forcedFumbleAsDefense`, `halfSackYardsAsDefense`, `sackYardsAsDefense`, `safetyAsDefense`, `tackleForALoss`, `tackleForALossYardage`, `interceptionYards`, `fumbleRecoveryYards`, `penaltyYards`.

- **Low Skewness and Kurtosis**: 
  - `gameId`, `playId`, `nflId`, `wasInitialPassRusher`, `timeToPressureAsPassRusher`, `getOffTimeAsPassRusher`.

### Correlations
- **Strong Correlations**:
  - `hadRushAttempt` and `rushingYards` (0.569)
  - `hadDropback` and `passingYards` (0.704)
  - `hadPassReception` and `receivingYards` (0.742)
  - `hadPassReception` and `wasTargettedReceiver` (0.816)
  - `receivingYards` and `yardageGainedAfterTheCatch` (0.739)
  - `hadInterception` and `interceptionYards` (0.597)

- **Negative Correlations**:
  - `sackYardsAsOffense` and `fumbles` (-0.207)
  - `halfSackYardsAsDefense` and `forcedFumbleAsDefense` (-0.058)
  - `sackYardsAsDefense` and `quarterbackHit` (-0.529)
  - `sackYardsAsDefense` and `safetyAsDefense` (-0.023)

### Outliers
- **High Outlier Counts**:
  - `soloTackle` (8,809 outliers, 2.48%)
  - `wasTargettedReceiver` (8,372 outliers, 2.36%)
  - `hadRushAttempt` (6,788 outliers, 1.91%)
  - `rushingYards` (6,229 outliers, 1.76%)
  - `hadDropback` (6,233 outliers, 1.76%)
  - `passingYards` (5,529 outliers, 1.56%)
  - `hadPassReception` (5,625 outliers, 1.59%)
  - `receivingYards` (5,538 outliers, 1.56%)

- **Low Outlier Counts**:
  - `safetyAsDefense` (4 outliers, 0.0011%)
  - `fumbleOutOfBounds` (19 outliers, 0.0054%)
  - `pff_secondaryDefensiveCoverageMatchupNflId` (1 outlier, 0.00028%)

### Key Insights
1. **High Missing Values**: Several columns have a significant percentage of missing values, which may require imputation or removal depending on their importance.
2. **Strong Correlations**: Key performance metrics like `rushingYards`, `passingYards`, and `receivingYards` are strongly correlated with their respective attempts, indicating consistency in the data.
3. **Outliers**: The presence of outliers in critical columns like `soloTackle`, `wasTargettedReceiver`, and `hadRushAttempt` suggests variability in player performance, which could be important for further analysis.
4. **Skewness and Kurtosis**: Many columns exhibit high skewness and kurtosis, indicating non-normal distributions, which may affect statistical analyses and model performance.
5. **Categorical Diversity**: Columns like `teamAbbr` and `penaltyNames` have a diverse set of unique values, which could be useful for categorical analysis.

### Recommendations
- **Data Cleaning**: Address the high percentage of missing values through imputation or removal.
- **Outlier Treatment**: Consider handling outliers to improve the robustness of statistical analyses and models.
- **Feature Engineering**: Leverage strong correlations to create new features that could enhance predictive models.
- **Distribution Analysis**: Apply transformations to normalize highly skewed distributions for better model performance.

This EDA summary provides a solid foundation for further data analysis and model building.

## Games EDA

In [17]:
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,9/8/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,9/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,9/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,9/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,9/11/2022,13:00:00,CIN,PIT,20,23


In [18]:
report = sv.analyze(games, pairwise_analysis="off")
report.show_html(filepath='/kaggle/working/games_reports.html', open_browser=False)

                                             |          | [  0%]   00:00 -> (? left)

Report /kaggle/working/games_reports.html was generated.


In [19]:
summary = eda_summary(games)
summary_json = json.dumps(summary, indent=4, default=str)
prompt = ChatPromptTemplate.from_template(template_eda)
chain = prompt | model | StrOutputParser()
result = chain.invoke(summary_json)
display(Markdown(result))

Based on the provided Exploratory Data Analysis (EDA) summary, here are the key insights and observations:

### General Overview
1. **Dataset Size**: The dataset contains 136 rows and 9 columns.
2. **Missing Values**: There are no missing values in the dataset, which is excellent for analysis as it means no imputation or data cleaning is required in this regard.

### Data Types
- The dataset includes a mix of integer (`int64`) and object (string) data types.
- Columns like `gameId`, `season`, `week`, `homeFinalScore`, and `visitorFinalScore` are numerical.
- Columns like `gameDate`, `gameTimeEastern`, `homeTeamAbbr`, and `visitorTeamAbbr` are categorical.

### Missing Values
- As mentioned, there are no missing values in any of the columns, which simplifies the preprocessing steps.

### Numerical Summary
- The numerical summary section is empty, indicating that specific statistics like mean, standard deviation, min, max, and quartiles were not provided. This information would typically be useful for understanding the distribution and central tendency of the numerical variables.

### Categorical Summary
- **`gameDate`**: 27 unique values, suggesting games span across 27 different dates.
- **`gameTimeEastern`**: 8 unique values, indicating games are played at 8 different times.
- **`homeTeamAbbr` and `visitorTeamAbbr`**: Both have 32 unique values, which likely correspond to the number of teams in the league.

### Skewness and Kurtosis
- **`gameId`**: Slightly positively skewed (0.163) and platykurtic (-0.805), indicating a relatively flat distribution.
- **`season`**: Both skewness and kurtosis are 0, suggesting a perfectly symmetrical distribution, which is unusual and might need verification.
- **`week`**: Slightly positively skewed (0.078) and platykurtic (-1.209), indicating a flat distribution.
- **`homeFinalScore`**: Positively skewed (0.442) and mesokurtic (0.221), suggesting a slight right tail and a normal-like peak.
- **`visitorFinalScore`**: Slightly positively skewed (0.230) and almost mesokurtic (-0.013), indicating a slight right tail and a normal-like peak.

### Correlations
- **`gameId` and `week`**: High positive correlation (0.942), suggesting that `gameId` increases with `week`.
- **`homeFinalScore` and `visitorFinalScore`**: Low positive correlation (0.151), indicating a weak relationship between the scores of home and visitor teams.
- **`season`**: All correlations are NaN, which might indicate that `season` has a constant value or there is an issue with the data.

### Outlier Summary
- **`homeFinalScore`**: 2 outliers (1.47% of the data), which is relatively low.
- **`visitorFinalScore`**: 1 outlier (0.74% of the data), also relatively low.
- No outliers in `gameId`, `season`, and `week`.

### Key Insights
1. **Data Completeness**: The dataset is complete with no missing values, which is ideal for analysis.
2. **Data Distribution**: Most numerical columns have slight skewness and kurtosis, indicating relatively normal distributions with some deviations.
3. **Correlations**: The high correlation between `gameId` and `week` suggests a sequential relationship. The weak correlation between scores indicates that the scores of home and visitor teams are relatively independent.
4. **Outliers**: There are a few outliers in the final scores, which might need further investigation to understand the context (e.g., unusually high or low scores).
5. **Categorical Diversity**: The categorical columns have a reasonable number of unique values, indicating diversity in game dates, times, and teams.

### Recommendations
1. **Numerical Summary**: Obtain and analyze the missing numerical summary statistics to better understand the central tendencies and spread of the numerical data.
2. **Season Data**: Verify the `season` data to ensure it is correctly recorded, as the NaN correlations are unusual.
3. **Outlier Analysis**: Investigate the outliers in the final scores to determine if they are data entry errors or legitimate values.
4. **Further Analysis**: Conduct additional analyses such as time series analysis on `gameDate` and `gameTimeEastern`, and performance analysis of teams based on `homeTeamAbbr` and `visitorTeamAbbr`.

This summary provides a comprehensive overview of the dataset's structure and initial findings, setting the stage for more detailed analysis.

## Play EDA

In [20]:
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,...,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone
1,2022091809,3698,(2:13) (Shotgun) J.Burrow pass short right to ...,4,1,10,CIN,DAL,CIN,8,...,4,0.002847,-0.002847,-0.240509,True,NaN,NaN,0,Quarters,Zone
2,2022103004,3146,(2:00) (Shotgun) D.Mills pass short right to D...,4,3,12,HOU,TEN,HOU,20,...,6,0.000205,-0.000205,-0.218480,True,NaN,NaN,0,Quarters,Zone
3,2022110610,348,(9:28) (Shotgun) P.Mahomes pass short left to ...,1,2,10,KC,TEN,TEN,23,...,4,-0.001308,0.001308,-0.427749,True,NaN,NaN,0,Quarters,Zone
4,2022102700,2799,(2:16) (Shotgun) L.Jackson up the middle to TB...,3,2,8,BAL,TB,TB,27,...,-1,0.027141,-0.027141,-0.638912,False,MAN,READ OPTION,0,Cover-1,Man


In [21]:
report = sv.analyze(plays, pairwise_analysis="off")
report.show_html(filepath='/kaggle/working/plays_reports.html', open_browser=False)

                                             |          | [  0%]   00:00 -> (? left)

Report /kaggle/working/plays_reports.html was generated.


In [22]:
summary = eda_summary(plays)
summary_json = json.dumps(summary, indent=4, default=str)
prompt = ChatPromptTemplate.from_template(template_eda)
chain = prompt | model | StrOutputParser()
result = chain.invoke(summary_json)
display(Markdown(result))

The provided Exploratory Data Analysis (EDA) summary offers a comprehensive overview of a dataset with 16,124 rows and 50 columns. Here are the key insights and observations:

### General Overview
- **Total Rows and Columns**: The dataset contains 16,124 rows and 50 columns.
- **Missing Values**: There are 153,703 missing values, which account for approximately 19.07% of the total data. This is a significant amount and may require imputation or removal of certain columns/rows depending on the analysis.

### Data Types
- **Data Types Distribution**:
  - **Numerical**: 24 columns (int64 and float64)
  - **Categorical**: 24 columns (object and bool)
  - **Boolean**: 2 columns (bool)

### Missing Values Analysis
- **Columns with No Missing Values**: 24 columns have no missing values, including key columns like `gameId`, `playId`, `quarter`, `down`, `yardsToGo`, etc.
- **Columns with Significant Missing Values**:
  - `penaltyYards` (97.62%)
  - `timeToSack` (96.23%)
  - `pff_runConceptSecondary` (82.50%)
  - `rushLocationType` (57.90%)
  - `qbSneak` (57.90%)
  - `targetX` and `targetY` (48.05%)
  - `passLength` (45.88%)
  - `timeToThrow` (46.01%)
  - `timeInTackleBox` (44.70%)
  - `passResult` (39.62%)
  - `dropbackType` (35.99%)
  - `dropbackDistance` (37.00%)
  - `passLocationType` (42.25%)
  - `passTippedAtLine` (42.10%)
  - `unblockedPressure` (39.50%)
  - `qbSpike` (42.10%)
  - `pff_runConceptPrimary` (43.74%)

### Categorical Summary
- **High Cardinality**: `playDescription` has 16,122 unique values, indicating almost every play is unique.
- **Moderate Cardinality**: Columns like `possessionTeam`, `defensiveTeam`, and `yardlineSide` have 32 unique values each, corresponding to the number of teams.
- **Low Cardinality**: Columns like `playNullifiedByPenalty` (1 unique value), `offenseFormation` (7 unique values), `receiverAlignment` (11 unique values), `passResult` (5 unique values), etc.

### Skewness and Kurtosis
- **Highly Skewed**: Columns like `qbKneel` (skewness: 9.73), `prePenaltyYardsGained` (skewness: 2.49), `yardsGained` (skewness: 2.23), `pff_runPassOption` (skewness: 2.64).
- **High Kurtosis**: Columns like `qbKneel` (kurtosis: 92.76), `prePenaltyYardsGained` (kurtosis: 10.98), `yardsGained` (kurtosis: 10.88), `homeTeamWinProbabilityAdded` and `visitorTeamWinProbilityAdded` (kurtosis: 30.01).

### Correlations
- **Strong Correlations**:
  - `playId` and `quarter` (0.96)
  - `preSnapHomeScore` and `playId` (0.71)
  - `preSnapVisitorScore` and `playId` (0.72)
  - `preSnapHomeScore` and `preSnapVisitorScore` (0.51)
  - `timeToThrow` and `timeInTackleBox` (0.87)
  - `dropbackDistance` and `timeToThrow` (0.48)
  - `dropbackDistance` and `timeInTackleBox` (0.49)
- **Negative Correlations**:
  - `preSnapHomeTeamWinProbability` and `preSnapVisitorTeamWinProbability` (-1.0)
  - `homeTeamWinProbabilityAdded` and `visitorTeamWinProbilityAdded` (-1.0)

### Outlier Analysis
- **Columns with High Outlier Counts**:
  - `homeTeamWinProbabilityAdded` and `visitorTeamWinProbilityAdded` (11.47%)
  - `pff_runPassOption` (10.13%)
  - `yardsGained` (5.76%)
  - `prePenaltyYardsGained` (5.41%)
  - `expectedPointsAdded` (5.57%)
- **Columns with No Outliers**: `gameId`, `playId`, `quarter`, `yardlineNumber`, `absoluteYardlineNumber`, `preSnapHomeTeamWinProbability`, `preSnapVisitorTeamWinProbability`, `expectedPoints`, `targetX`, `targetY`.

### Key Insights
1. **High Missing Values**: Several columns have a high percentage of missing values, which could impact the analysis. Strategies like imputation, removal, or using algorithms that handle missing data are necessary.
2. **High Cardinality in `playDescription`**: This column has almost unique values for each row, indicating detailed play descriptions.
3. **Strong Correlations**: Some columns are highly correlated, which could be useful for predictive modeling but may also indicate multicollinearity issues.
4. **Outliers**: Several columns have a significant number of outliers, which could affect the analysis and may need to be addressed through techniques like transformation or robust statistical methods.
5. **Skewness and Kurtosis**: Many columns exhibit high skewness and kurtosis, indicating non-normal distributions, which may require transformation for certain analyses.

### Recommendations
- **Handle Missing Values**: Consider imputation methods or removal of columns/rows with excessive missing values.
- **Address Outliers**: Use robust statistical methods or transformations to handle outliers.
- **Transform Skewed Data**: Apply transformations (e.g., log, square root) to normalize highly skewed data.
- **Feature Engineering**: Create new features or aggregate existing ones to reduce dimensionality and improve model performance.
- **Correlation Analysis**: Address multicollinearity by removing or combining highly correlated features.

This EDA summary provides a solid foundation for further analysis and modeling, highlighting areas that need attention and potential strategies for handling data issues.